In [2]:
import tensorflow as tf
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
import numpy as np
import os
import xml.etree.ElementTree as ET
import cv2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from google.colab.patches import cv2_imshow  # Import cv2_imshow for displaying images
import time

# Define the path to the directory containing the JPG and XML files
train_directory_path = r'/content/drive/MyDrive/thesis/dataset/roboflow-cnn/train'
valid_directory_path = r'/content/drive/MyDrive/thesis/dataset/roboflow-cnn/valid'

# Define the batch size
batch_size = 32

# Define the input shape of the model
input_shape = (224, 224, 3)

# Load the ResNet152V2 model
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=input_shape)

# Add a custom output layer for bounding box regression (4 values)
x = base_model.output
x = Flatten()(x)
x = Dense(4, activation='linear')(x)  # 4 values for xmin, ymin, xmax, ymax
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])
save_best = tf.keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/thesis/cnn_models/ResNet152V2Modelv1.h5", monitor='accuracy', save_best_only=True, verbose=1)

# Define a callback to track and print total training time and highest accuracy
class TimingCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        self.start_time = None
        self.total_time = 0.0
        self.highest_train_acc = 0.0
        self.highest_val_acc = 0.0
        self.highest_train_epoch = 0
        self.highest_val_epoch = 0

    def on_epoch_begin(self, epoch, logs=None):
        self.start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.start_time
        self.total_time += epoch_time
        print(f'Total training time for epoch {epoch + 1}: {epoch_time:.2f} seconds')

        train_acc = logs['accuracy']
        val_acc = logs['val_accuracy']

        if train_acc > self.highest_train_acc:
            self.highest_train_acc = train_acc
            self.highest_train_epoch = epoch + 1

        if val_acc > self.highest_val_acc:
            self.highest_val_acc = val_acc
            self.highest_val_epoch = epoch + 1

        print(f'Highest training accuracy: {self.highest_train_acc:.4f} at epoch {self.highest_train_epoch}')
        print(f'Highest validation accuracy: {self.highest_val_acc:.4f} at epoch {self.highest_val_epoch}')

# Define a data generator that yields batches of images and their corresponding annotations
def data_generator(directory_path, batch_size, input_shape):
    while True:
        data = []
        train_annotation = []
        for filename in os.listdir(directory_path):
            if filename.endswith('.jpg'):
                # Extract the file path of the JPG image
                image_path = os.path.join(directory_path, filename)
                # Construct the file path of the corresponding XML annotation file
                xml_path = os.path.join(directory_path, filename[:-4] + '.xml')

                # Parse the XML annotation file
                tree = ET.parse(xml_path)
                root = tree.getroot()
                # Check if the bndbox element is present
                bndbox = root.find('object/bndbox')
                if bndbox is not None:
                    # Extract the bounding box coordinates
                    xmin = int(bndbox.find('xmin').text)
                    xmax = int(bndbox.find('xmax').text)
                    ymin = int(bndbox.find('ymin').text)
                    ymax = int(bndbox.find('ymax').text)
                    # Normalize the bounding box coordinates to the range [0, 1]
                    train_annotation.append([xmin / input_shape[0], ymin / input_shape[1],
                                             xmax / input_shape[0], ymax / input_shape[1]])

                    # Load the image and preprocess it
                    image = cv2.resize(cv2.imread(image_path), input_shape[:2])
                    image = preprocess_input(image)
                    data.append(image)

                    # Yield the batch when the data and train_annotation lists are of length batch_size
                    if len(data) == batch_size:
                        yield np.array(data), np.array(train_annotation)
                        data = []
                        train_annotation = []

        # Yield the last batch if it is smaller than batch_size
        if len(data) > 0:
            yield np.array(data), np.array(train_annotation)

# Train the model with the timing callback
timing_callback = TimingCallback()
history = model.fit(data_generator(train_directory_path, batch_size, input_shape),
                    steps_per_epoch=len(os.listdir(train_directory_path)) // batch_size,
                    epochs=25,
                    verbose=1,
                    validation_data=data_generator(valid_directory_path, batch_size, input_shape),
                    validation_steps=len(os.listdir(valid_directory_path)) // batch_size,
                    callbacks=[save_best, timing_callback])


Epoch 1/25
123/123 [==============================] - ETA: 0s - loss: 0.6943 - accuracy: 0.5843
Epoch 1: accuracy improved from -inf to 0.58429, saving model to /content/drive/MyDrive/thesis/cnn_models/ResNet152V2Modelv1.h5
Total training time for epoch 1: 228.27 seconds
Highest training accuracy: 0.5843 at epoch 1
Highest validation accuracy: 0.6184 at epoch 1
123/123 [==============================] - 228s 1s/step - loss: 0.6943 - accuracy: 0.5843 - val_loss: 0.4648 - val_accuracy: 0.6184
Epoch 2/25
123/123 [==============================] - ETA: 0s - loss: 0.0607 - accuracy: 0.8692
Epoch 2: accuracy improved from 0.58429 to 0.86918, saving model to /content/drive/MyDrive/thesis/cnn_models/ResNet152V2Modelv1.h5
Total training time for epoch 2: 134.05 seconds
Highest training accuracy: 0.8692 at epoch 2
Highest validation accuracy: 0.6184 at epoch 1
123/123 [==============================] - 134s 1s/step - loss: 0.0607 - accuracy: 0.8692 - val_loss: 0.2567 - val_accuracy: 0.6166
Epoch

In [12]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt

# Define a data generator that yields batches of test images and their corresponding annotations
def test_data_generator(directory_path, batch_size, input_shape):
    while True:
        data = []
        test_annotation = []
        for filename in os.listdir(directory_path):
            if filename.endswith('.jpg'):
                # Extract the file path of the JPG image
                image_path = os.path.join(directory_path, filename)
                # Construct the file path of the corresponding XML annotation file
                xml_path = os.path.join(directory_path, filename[:-4] + '.xml')

                # Parse the XML annotation file
                tree = ET.parse(xml_path)
                root = tree.getroot()
                # Check if the bndbox element is present
                bndbox = root.find('object/bndbox')
                if bndbox is not None:
                    # Extract the bounding box coordinates
                    xmin = int(bndbox.find('xmin').text)
                    xmax = int(bndbox.find('xmax').text)
                    ymin = int(bndbox.find('ymin').text)
                    ymax = int(bndbox.find('ymax').text)
                    # Normalize the bounding box coordinates to the range [0, 1]
                    test_annotation.append([xmin / input_shape[0], ymin / input_shape[1],
                                            xmax / input_shape[0], ymax / input_shape[1]])

                    # Load the image
                    image = cv2.resize(cv2.imread(image_path), input_shape[:2])
                    data.append(image)

                    # Yield the batch when the data and test_annotation lists are of length batch_size
                    if len(data) == batch_size:
                        yield np.array(data), np.array(test_annotation)
                        data = []
                        test_annotation = []

        # Yield the last batch if it is smaller than batch_size
        if len(data) > 0:
            yield np.array(data), np.array(test_annotation)


# Load the saved model
model_path = '/content/drive/MyDrive/thesis/cnn_models/ResNet152V2Modelv1.h5'
Model = tf.keras.models.load_model(model_path)

# Define the path to the directory containing the test JPG files
test_directory_path = r'/content/drive/MyDrive/thesis/dataset/roboflow-cnn/test'

# Assuming you have 32 test samples, change this to the actual number of test samples
num_test_samples = 32

# Initialize the data generator for the test set
batch_size = 1  # Set the batch size for testing (you can adjust it as needed)
input_shape = (224, 224)  # Set the input shape based on the model's input requirements
# Get the list of image file names in the test directory
test_image_files = [filename for filename in os.listdir(test_directory_path) if filename.endswith('.jpg')]

test_generator = test_data_generator(test_directory_path, batch_size, input_shape)

# Loop through the test dataset and make predictions
for i in range(num_test_samples):
    images, annotations = next(test_generator)

    predictions = Model.predict(images)

    image = images[0].astype('uint8')  # Assuming batch size is 1
    annotation = annotations[0]
    pred = predictions[0]

    # Convert the normalized coordinates to pixel values for 224x224 input shape
    xmin, ymin, xmax, ymax = [int(anno * input_shape[i // 2]) for i, anno in enumerate(annotation)]
    xmin1, ymin1, xmax1, ymax1 = [int(p * input_shape[i // 2]) for i, p in enumerate(pred)]

    # Determine the class label based on the prediction score
    class_label = "Violence" if pred[0] > 0.5 else "NonViolence"

    # Set the color of the bounding box based on the class label
    box_color = (0, 255, 0)  # Green color for NonViolence
    if class_label == "Violence":
        box_color = (255, 0, 0)  # Red color for Violence

    cv2.rectangle(image, (xmin1, ymin1), (xmax1, ymax1), box_color, 2)
    cv2.putText(image, class_label, (xmin1, ymin1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 1)
    plt.imshow(image)
    plt.title('Prediction from ResNet152V2Modelv1')
    plt.show()

    box1 = [xmin, ymin, xmax, ymax]  # coordinates of the ground truth bounding box
    box2 = [xmin1, ymin1, xmax1, ymax1]  # coordinates of the predicted bounding box

    # Function to calculate the Intersection over Union (IoU)
    def calculate_iou(box1, box2):
        x5 = max(box1[0], box2[0])
        y5 = max(box1[1], box2[1])
        x6 = min(box1[2], box2[2])
        y6 = min(box1[3], box2[3])

        # calculate the area of intersection
        inter_area = max(0, x6 - x5) * max(0, y6 - y5)

        # calculate the area of union
        box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
        box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union_area = box1_area + box2_area - inter_area

        # calculate the IoU
        iou = inter_area / union_area
        return iou

    iou = calculate_iou(box1, box2)
    print(f"IoU for image {i+1}: {iou}")

Output hidden; open in https://colab.research.google.com to view.